# **Song comparison**

In [ ]:
#! pip install panns-inference
#!pip install chromadb

In [1]:
import librosa
import panns_inference
from panns_inference import AudioTagging, SoundEventDetection, labels
import numpy as np
import pandas as pd
from langchain_community.vectorstores import FAISS
import chromadb

## **Convert the songs into embeddings using PANNS**

In [2]:
#define the 3 .mp3 files
songs = ["songs/1.mp3","songs/2.mp3","songs/3.mp3", "songs/acrossuni_1.mp3", "songs/acrossuni_2.mp3"]

In [3]:
## function to convert song into embedding and save the output as.npy file

def convert_song_embedding(location_of_song, song_number):
    
    (audio, _) = librosa.core.load(location_of_song, sr=32000, mono=True)
    audio = audio[None, :]  # (batch_size, segment_samples)
    
    at = AudioTagging(checkpoint_path=None, device='cuda')
    (clipwise_output, embedding) = at.inference(audio)
    
    #save array
    np.save(f'song_{song_number}.npy', embedding)
    
    print("--next song--")


In [4]:
#loop through the 3 songs and convert to embeddings

song_number = 1

for location_of_song in songs:
    
    print(location_of_song)
        
    convert_song_embedding(location_of_song, song_number)
    
    song_number +=1
    


songs/1.mp3
Checkpoint path: /home/datascience/panns_data/Cnn14_mAP=0.431.pth
Using CPU.
--next song--
songs/2.mp3
Checkpoint path: /home/datascience/panns_data/Cnn14_mAP=0.431.pth
Using CPU.
--next song--
songs/3.mp3
Checkpoint path: /home/datascience/panns_data/Cnn14_mAP=0.431.pth
Using CPU.
--next song--
songs/acrossuni_1.mp3
Checkpoint path: /home/datascience/panns_data/Cnn14_mAP=0.431.pth
Using CPU.
--next song--
songs/acrossuni_2.mp3
Checkpoint path: /home/datascience/panns_data/Cnn14_mAP=0.431.pth
Using CPU.
--next song--


## **Check similarity**

In [5]:
# Load the saved arrays
emb1 = np.load('song_1.npy')
emb2 = np.load('song_2.npy')
emb3 = np.load('song_3.npy')
emb1_across = np.load('song_4.npy')
emb2_across = np.load('song_5.npy')

In [6]:
def get_cosine_similarity(vector1, vector2):
    """
    Get cosine similarity value between 2 vectors embeddings
    """
    vector1 = np.array(vector1.tolist()[0])
    vector2 = np.array(vector2.tolist()[0])
    cosine_similarity = np.dot(vector1, vector2) / (
        np.linalg.norm(vector1) * np.linalg.norm(vector2)
    )

    return cosine_similarity

In [7]:
def similarsound(emb1, emb2):
    """
    Cosine Similarity analysis
    """
    cosine = get_cosine_similarity(emb1, emb2)

    if cosine == 1:
        decision = "identical"
        color_code = "\033[1;31;34m"
    elif cosine >= 0.8:
        decision = "similar"
        color_code = "\033[1;31;32m"
    else:
        decision = "different"
        color_code = "\033[1;31;91m"

    print(f"{color_code}{decision.upper()} song (cosine similarity = {cosine})")


In [8]:
# review similarity scores
similarsound(emb1_across, emb3)

DIFFERENT song (cosine similarity = 0.6808329531694801)


## **Store embeddings in ChromaDB**

In [9]:
collection_name = "all_songs_11"

In [10]:
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name=collection_name)

In [11]:
#load embeddings and add them to vector db
# Load the saved arrays
emb1 = np.load('song_1.npy').tolist()[0]
emb2 = np.load('song_2.npy').tolist()[0]
emb3 = np.load('song_3.npy').tolist()[0]
emb1_across = np.load('song_4.npy').tolist()[0]
emb2_across = np.load('song_5.npy').tolist()[0]

collection.add(
    documents=songs,
    embeddings=[emb1,emb2,emb3,emb1_across,emb2_across],   #these are the loaded embeddings
    ids=["id1", "id2", "id3", "id4", "id5"])

In [12]:
from langchain.vectorstores import Chroma
langchainChroma = Chroma(client=chroma_client, collection_name=collection_name) 

print(langchainChroma._collection.count())

5


In [13]:
output = langchainChroma.similarity_search_by_vector_with_relevance_scores(emb2_across, k=10)

Number of requested results 10 is greater than number of elements in index 5, updating n_results = 5


In [14]:
output

[(Document(page_content='songs/acrossuni_2.mp3'), 0.0),
 (Document(page_content='songs/acrossuni_1.mp3'), 76.51272583007812),
 (Document(page_content='songs/2.mp3'), 100.62278747558594),
 (Document(page_content='songs/1.mp3'), 103.17691040039062),
 (Document(page_content='songs/3.mp3'), 117.47669982910156)]

In [15]:
df = pd.DataFrame(output, columns = ["song","similarity_score"])
df['ranking'] = df['similarity_score'].rank() 
df

,song,similarity_score,ranking
0,page_content='songs/acrossuni_2.mp3',0.000000,1.0
1,page_content='songs/acrossuni_1.mp3',76.512726,2.0
2,page_content='songs/2.mp3',100.622787,3.0
3,page_content='songs/1.mp3',103.176910,4.0
4,page_content='songs/3.mp3',117.476700,5.0


In [16]:

def process_file(fileobj): 
    file_path = fileobj.name 
    print(file_path)
    return check_similar_mps(file_path)


def check_similar_mps(file_path):
    
    print("Start creating embedding")
    
    #create as embedding and save
    convert_song_embedding(file_path, 100)
    
    #load
    emb100 = np.load('song_100.npy')
    
    print("Start cross check")
    #cross_check
    langchainChroma = Chroma(client=chroma_client, collection_name=collection_name) 
    output = langchainChroma.similarity_search_by_vector_with_relevance_scores(emb100, k=5)
    
    df = pd.DataFrame(output, columns = ["song","similarity_score"])
    df['ranking'] = df['similarity_score'].rank() 

    return df


In [ ]:
import gradio as gr

demo = gr.Interface(fn=process_file, inputs="file", outputs=gr.Dataframe())
demo.launch(share=True, debug=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://f464e98950db5bdc73.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/tmp/gradio/a4d92896db3ea055d2de65841841d0c43f6cd2d7/Across The Universe Remastered 2009.mp3
Start creating embedding
Checkpoint path: /home/datascience/panns_data/Cnn14_mAP=0.431.pth
Using CPU.
--next song--
Start cross check
/tmp/gradio/978889166a7dfda6c766f5f39a5acf0e2b234b61/Fiona Apple - Across the Universe Official HD Video.mp3
Start creating embedding
Checkpoint path: /home/datascience/panns_data/Cnn14_mAP=0.431.pth
Using CPU.
--next song--
Start cross check
